In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'http://www.yes24.com/Product/Goods/106711883'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
tag_name = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2'
books = soup.select(tag_name)
print(books)

[<h2 class="gd_name">진짜 쓰는 실무 엑셀</h2>]


In [3]:
name = books[0].text
print(name)

진짜 쓰는 실무 엑셀


In [3]:
# 데이터 수집 함수 정의
soup = BeautifulSoup(res.text, 'lxml')
def yes24DataReader(CategoryNumber, year, month):

    root_url = 'http://www.yes24.com'

    url_1 = 'http://www.yes24.com/24/category/bestseller?CategoryNumber='
    url_2 = '&sumgb=09&year='
    url_3 = '&month='
    url_4 = '&PageNumber='
    url_set = url_1 + CategoryNumber + url_2 + year + url_3 + month + url_4

    book_list=[]

    # 월 별 조회 시 최대 50쪽이지만, 간단한 설명을 위해 2쪽까지만 수집
    for i in range(1, 3):

        url = url_set + str(i)

        res = requests.post(url)
        soup = BeautifulSoup(res.text, 'html5lib')
        tag = '#category_layout > tbody > tr > td.goodsTxtInfo > p:nth-child(1) > a:nth-child(1)'
        books = soup.select(tag)

        # 수집 중인 페이지 번호 출력
        print('# Page', i)

        # 개별 도서 정보 수집
        for book in books:

            sub_url = root_url + book.attrs['href']
            sub_res = requests.post(sub_url)
            sub_soup = BeautifulSoup(sub_res.text, 'html5lib')

            tag_name = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2'
            tag_author = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a'
            tag_author2 = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth'
            tag_publisher = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a'
            tag_date = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_date'
         
            
            tag_price = '#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(3) > table > tbody > tr:nth-child(2) > td > span > em'
            tag_price2 = '#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(4) > table > tbody > tr:nth-child(2) > td > span > em'

            tag_page = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td'
            tag_weight = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td'
            tag_hor = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td'
            tag_ver = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td'
            tag_width = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td'
            tag_isbn13 = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td'
            tag_description = '#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt > div > textarea'
         


            # 기본적인 예외처리를 통한 데이터 수집
            name = sub_soup.select(tag_name)[0].text

            try:
                author = sub_soup.select(tag_author)[0].text
            except:
                author = sub_soup.select(tag_author2)[0].text.strip('\n').strip().replace(' 저','')


            publisher = sub_soup.select(tag_publisher)[0].text
            date = sub_soup.select(tag_date)[0].text.replace('년 ','-').replace('월 ','-').replace('일','')
            
        


            try:
                price = sub_soup.select(tag_price)[0].text.replace(',','')
            except:
                try:
                    price = sub_soup.select(tag_price2)[0].text.replace(',','')
                except:
                    price = ''

            page = sub_soup.select(tag_page)[0].text
            if '쪽' in page:
                if '확인' in page:
                    page = ''
                else :
                    page = page.split('|')[0].strip().replace('쪽','')
            else :
                page = ''

            weight = sub_soup.select(tag_weight)[0].text
            if 'g' in weight:
                weight = weight[:weight.find('g')].split('|')[1].strip()
            else :
                weight = ''

            hvw = sub_soup.select(tag_hor)[0].text
            if 'mm' in hvw:

                if hvw.split('|')[-1].strip().count('*')==2:

                    hor = hvw.split('|')[-1].strip().split('*')[0]
                    ver = hvw.split('|')[-1].strip().split('*')[1]
                    width = hvw.split('|')[-1].strip().split('*')[2].replace('mm','')

                elif hvw.split('|')[-1].strip().count('*')==1:

                    hor = hvw.split('|')[-1].strip().split('*')[0]
                    ver = hvw.split('|')[-1].strip().split('*')[1].replace('mm','')
                    width = ''

            else :
                hor = ''
                ver = ''
                width = ''

            try :
                isbn13 = sub_soup.select(tag_isbn13)[0].text
                if '확인' in isbn13:
                    isbn13 = ''
                else :
                    isbn13 = sub_soup.select(tag_isbn13)[0].text
            except :
                isbn13 = ''
                
            try:
                description = sub_soup.select(tag_description)[0].text
                description = description.replace('<br/>' ,'')  # <br/> 태그 제거
                cleanr = re.compile('<.*?>')
                description = re.sub(cleanr, '', description)
            except IndexError:
                description = ""
    




            book_list.append([name, author, publisher, date,
                               price, page,
                              weight, hor, ver, width, isbn13,description ])

            print('=========>', name)

    # 데이터프레임 컬럼명 지정
    colList = ['name',  'author', 'publisher', 'date',
                'price', 'page',
               'weight', 'hor', 'ver', 'width', 'isbn13','description']

    # 데이터프레임으로 변환
    df = pd.DataFrame(np.array(book_list), columns=colList)

    return df

In [11]:
import re
# 역사 카테고리 번호 : 001001010
# IT 카테고리 번호 : 001001003
# 여행 카테고리 번호 : 001001009
# 한국소설 카테고리 번호 : 001001046001
# 외국소설 카테고리 번호 : 001001046002
# 사회정치 카테고리 번호 : 001001022
# 경제경영 카테고리 번호 : 001001025
# 예술 카테고리 번호 : 001001007
# 어린이 카테고리 번호 : 001001016
# 에세이 카테고리 번호 : 001001047
CategoryNum='001001047'


# 2019년도
for year in range(2023, 2024):
    print('='*50)
    print('# Year', year)
    print('='*50)

    # 1월
    for month in range(1, 2):
        print('='*50)
        print('# Month', month)
        print('='*50)

        # 월 별 데이터 수집
        df = yes24DataReader(CategoryNum, str(year), str(month))

        # 월 별로 수집된 데이터를 CSV 형식 파일로 저장
        df.to_csv(str(year)+'_'+str(month)+'_'+str(CategoryNum)+'.csv', index=False, encoding='UTF-8')

# Year 2023
# Month 1
# Page 1
=========> 모든 것은 기본에서 시작한다
=========> 물고기는 존재하지 않는다
=========> 너의 하루가 따숩길 바라
=========> 인생의 역사
=========> 기분이 태도가 되지 말자
=========> 불안
=========> 잘될 수밖에 없는 너에게
=========> 천문학자는 별을 보지 않는다
=========> 보이지 않는 곳에서 애쓰고 있는 너에게
=========> 축구를 하며 생각한 것들 (리커버 에디션)
=========> 빅터 프랭클의 죽음의 수용소에서
=========> 기분을 관리하면 인생이 관리된다
=========> 너의 안부
=========> 인듀어런스
=========> 생에 감사해
=========> 사람을 얻는 지혜 
=========> Hoy 
=========> 망그러진 만화
=========> 잘했고 잘하고 있고 잘 될 것이다 (스페셜 리미티드 에디션)
=========> 내 안의 어린아이에게 세트
# Page 2
=========> 100세 철학자의 행복론
=========> 2023년 문재인 일력
=========> 여행의 이유
=========> 착한 아이 버리기 
=========> 나는 나로 살기로 했다
=========> 나는 당신이 행복했으면 좋겠습니다
=========> 내일, 내가 다시 좋아지고 싶어
=========> 아주 오랜만에 행복하다는 느낌
=========> 새는 날아가면서 뒤돌아보지 않는다
=========> H마트에서 울다
=========> 나는 나를 응원합니다
=========> 어떤 고독은 외롭지 않다
=========> 하루 한 문장, 고전 명작 일력 
=========> 빅터 프랭클의 죽음의 수용소에서
=========> 상대적이며 절대적인 고양이 백과사전
=========> 작은 별이지만 빛나고 있어 (20만 부 기념 한정판 에디션)
=========> 저 별은 모두 당신을 위해 빛나고 있다
==

In [52]:
import re
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import requests

def yes24DataReader(CategoryNum, year, month):
    url = f'http://www.yes24.com/24/category/bestseller?CategoryNumber={CategoryNum}&sumgb=06&year={year}&month={month}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')

    books = soup.select('ol > li')
    book_list = []

    for book in books:
        # 도서 제목 수집
        title = book.select('a')[0]['title']

        # 도서 이미지 링크 수집
        img_src = book.select('a > img')[0]['src']

        # 도서 설명 페이지로 이동
        description_link = book.select('a')[0]['href']
        sub_res = requests.get(description_link)
        sub_soup = BeautifulSoup(sub_res.text, 'lxml')

        # 도서 설명 수집
        try:
            description = sub_soup.select('div.infoWrap_txt')[0].text.strip()
            description = re.sub('\s+', ' ', description) # 공백 제거
            description = description.replace('<br/>' ,'')  # <br/> 태그 제거
        except IndexError:
            description = ""

        # 도서 정보를 딕셔너리에 저장
        book_dict = {
            'Title': title,
            'Image Link': img_src,
            'Description': description
        }

        # 도서 이미지 저장
        image_save(img_src, title)

        # 딕셔너리를 리스트에 추가
        book_list.append(book_dict)

    # 리스트를 데이터프레임으로 변환
    import pandas as pd
    df = pd.DataFrame(book_list)

    return df

def image_save(src, title):
    savename = '/Users/woongcor/images/' + title + ".jpg"
    urlretrieve(src, savename)
    print(title, '이미지 저장됨')

# 역사 카테고리 번호 : 001001010
CategoryNum='001001003'

# 2019년도
for year in range(2023, 2024):
    print('='*50)
    print('# Year', year)
    print('='*50)

    # 9월
    for month in range(1, 3):
        print('='*50)
        print('# Month', month)
        print('='*50)

        # 월 별 데이터 수집
        df = yes24DataReader(CategoryNum, str(year), str(month))

        # 월 별로 수집된 데이터를 CSV 형식 파일로 저장
        df.to_csv(str(year)+'_'+str(month)+'_'+str(CategoryNum)+'.csv', index=False, encoding='UTF-8')


# Year 2023
# Month 1
# Month 2


In [56]:
# 지피티 센세
from bs4 import BeautifulSoup 
import urllib
import re

# 역사 카테고리 번호 : 001001010
CategoryNum='001001003'

# 검색할 사이트
root_url = "http://www.yes24.com/24/category/bestseller?CategoryNumber=" + CategoryNum + "&sumgb=06"

# HTML 헤더
head = {'User-Agent' : "Magic Browser"}

# 2019년도
for year in range(2023, 2024):
    print('='*50)
    print('# Year', year)
    print('='*50)

    # 1월부터 2월까지
    for month in range(1, 3):
        print('='*50)
        print('# Month', month)
        print('='*50)

        # URL 조합하기
        url = root_url + '&year=' + str(year) + '&month=' + str(month)

        # 요청보내기
        request = urllib.request.Request(url, headers = head) 

        # 응답 받기
        response = urllib.request.urlopen(request)

        # 응답을 Beautifulsoup 패키징화하기
        soup = BeautifulSoup(response.read().decode('euc-kr','replace'))

        # 월 별 데이터 수집
        df = yes24DataReader(CategoryNum, str(year), str(month))
        
        # 월 별로 수집된 데이터를 CSV 형식 파일로 저장
        df.to_csv(str(year)+'_'+str(month)+'_'+str(CategoryNum)+'.csv', index=False, encoding='UTF-8') 





# Year 2023
# Month 1
# Page 1
=========> 진짜 쓰는 실무 엑셀
=========> 혼자 공부하는 파이썬
=========> 제품의 탄생
=========> 혼자 공부하는 머신러닝+딥러닝
=========> 실무에 바로 쓰는 일잘러의 보고서 작성법
=========> 퇴근 후 스마트스토어로 투잡하기
=========> Do it! 점프 투 파이썬
=========> 만들면서 배우는 파이썬과 40개의 작품들
=========> 회사에서 바로 통하는 실무 엑셀+파워포인트+워드&한글 
=========> 비전공자도 이해할 수 있는 AI 지식
=========> 혼자 공부하는 컴퓨터 구조+운영체제
=========> 회사는 엑셀을 가르쳐주지 않아요
=========> 비전공자를 위한 이해할 수 있는 IT 지식 
=========> 개발자 원칙
=========> 유튜브 영상 편집을 위한 프리미어 프로
=========> Clean Code 클린 코드
=========> NestJS로 배우는 백엔드 프로그래밍
=========> 업무와 일상을 정리하는 새로운 방법 노션 Notion
=========> 코딩 자율학습 나도코딩의 C 언어 입문
=========> 구글 엔지니어는 이렇게 일한다
# Page 2
=========> 혼자 해도 프로 작가처럼 잘 그리는 아이패드 드로잉 with 프로크리에이트
=========> 혼자 공부하는 C 언어
=========> 맛있는 디자인 포토샵&일러스트레이터 CC 2022
=========> Java의 정석
=========> 1일 1로그 100일 완성 IT 지식
=========> 국내 최다 이모티콘 승인 작가 씨엠제이가 알려주는 승인율 99.9% 이모티콘 만들기
=========> 된다! 네이버 블로그 상위 노출
=========> 면접을 위한 CS 전공지식 노트
=========> Do it! HTML+CSS+자바스크립트 웹 표준의 정석
=========> 혼자 공부하는 SQL
=========>

In [4]:
from bs4 import BeautifulSoup 
import urllib

# 검색할 사이트
root_url = "http://www.yes24.com/24/category/bestseller?CategoryNumber=001001003&sumgb=09"

# 원하는 연도와 월의 범위 설정
years = range(2023, 2024)
months = range(1, 6)

# URL 조합 및 스크래핑 수행
for year in years:
    for month in months:
        url = root_url + f"&year={year}&month={month}"
        
        # HTML 헤더
        head = {'User-Agent' : "Magic Browser"}

        # 요청보내기
        request = urllib.request.Request(url, headers=head) 

        # 응답 받기
        response = urllib.request.urlopen(request)

        # 응답을 Beautifulsoup 패키징화하기
        soup = BeautifulSoup(response.read().decode('euc-kr','replace'))

        # 이후에 웹 스크래핑 코드 작성
        # 예를 들어, 베스트셀러 도서 제목 추출
        book_titles = soup.select('ol li p.copy > a')
        for title in book_titles:
            print(title.text.strip())

            

In [5]:
import os # 시스템 함수

# 이미지를 저장할 디렉토리
directory_path = "book_image"

# 디렉토리가 존재하지 않을 시 자동 생성
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [6]:
# 도서 이미지를 포함하고 있는 태그
contents = soup.find_all('div', {'class':'goodsImgW'})

for content in contents:
    
    # 각 도서를 의미하는 <a> 태그
    book_image = content.find_all('a')

    # 태그안에 있는 모든 이미지
    for i in range(0, len(book_image)):
        if i % 2 ==0:
            # 이미지 URL
            image_url = book_image[i].find_all('img')[0].get("src")
            print(image_url)

            # 이미지 이름
            image_name = book_image[i].find_all('img')[0].get("alt")
            print(image_name)

            urllib.request.urlretrieve(image_url, directory_path+"/"+image_name+".jpg")

http://image.yes24.com/goods/117936860/S
진짜 챗GPT 활용법
http://image.yes24.com/goods/106711883/S
진짜 쓰는 실무 엑셀
http://image.yes24.com/goods/117937260/S
챗GPT와 업무자동화
http://image.yes24.com/goods/117603800/S
챗GPT와 글쓰기
http://image.yes24.com/goods/109625396/S
혼자 공부하는 파이썬
http://image.yes24.com/goods/107490270/S
만들면서 배우는 파이썬과 40개의 작품들
http://image.yes24.com/goods/112941801/S
회사는 엑셀을 가르쳐주지 않아요
http://image.yes24.com/goods/74419916/S
Do it! 점프 투 파이썬
http://image.yes24.com/goods/118266847/S
Docs for Developers 기술 문서 작성 완벽 가이드
http://image.yes24.com/goods/96024871/S
혼자 공부하는 머신러닝+딥러닝
http://image.yes24.com/goods/111378840/S
혼자 공부하는 컴퓨터 구조+운영체제
http://image.yes24.com/goods/93068762/S
퇴근 후 스마트스토어로 투잡하기
http://image.yes24.com/goods/117024821/S
맛있는 디자인 포토샵&일러스트레이터 CC 2023
http://image.yes24.com/goods/109662792/S
회사에서 바로 통하는 실무 엑셀+파워포인트+워드&한글 
http://image.yes24.com/goods/95162490/S
실무에 바로 쓰는 일잘러의 보고서 작성법
http://image.yes24.com/goods/117971215/S
개발자를 위한 챗GPT 활용법
http://image.yes24.com/goods/11681152/S
Cle